# Fake News Prediction Project
### 1 Problem Definition
- Goal: Predict what is fake news and what is real news
### 2 Preprocessing
- Remove stop word
- Covert word to root word (stemming)
- Covert text to numerical

### 3 Splitting the datasets
 - `data`: dataset
 - `x` : data(feature)
 - `y` : target variable( 1: fake news, 0: real news)


### Import library

In [1]:
import numpy as np
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Show stop words in english. 
 #### Stop word is these words is not important in text

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Loading data using pandas

In [4]:
dataset = pd.read_csv('./data.csv')

In [5]:
dataset.shape

(20800, 5)

In [6]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


## Data pre-processing

#### Counting missing value in the dataset

In [7]:
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

#### Replacing null value to empty string

In [8]:
dataset = dataset.fillna('')

 #### Check again missing value

In [9]:
dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

### Merging columns author with news title together

In [10]:
dataset['content'] = dataset['author'] +" " + dataset['title']

In [11]:

#check again dataset
dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1,Howard Portnoy Iranian woman jailed for fictio...


#### Stemming:

   - Stemming is a process of reducing a word to its Root word 
  
   - Example: actor, actress, acting, ==> act(root word)

In [12]:
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    
    return ' '.join(stemmed_content)
    

In [14]:
dataset['content'] = dataset['content'].apply(stemming)


### Separating data & label 

In [15]:
X = dataset['content'].values
y = dataset['label'].values

### Converting text to numerical

In [16]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)


TfidfVectorizer()

In [17]:
X = vectorizer.transform(X)

### Slpiting data to traing and testing 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y ,random_state=2)

### Training model: Logistic Regression

In [19]:
model = LogisticRegression()

In [20]:
model.fit(X_train,y_train)

LogisticRegression()

### Evalution Model

In [21]:
# accuracy score of training data
training_prediction = model.predict(X_train)
accuracy_score(training_prediction,y_train)

0.9865985576923076

In [22]:
# accuracy score of testing data
testing_prediction = model.predict(X_test)
accuracy_score(testing_prediction,y_test)

0.9790865384615385

In [23]:
with open("./model.pkl", 'wb') as f:   
    pickle.dump(model, f)

In [24]:
with open("./TfidfVectorizer.pkl", 'wb') as f:   
    pickle.dump(vectorizer, f)